# Part 1 — The birthday paradox

Suppose you're in a room with 22 other people. What is the chance that any pair of people in the room have the same birthday? The answer will likely surprise you — it's much greater than most people think.

In this exercise we'll use simulations to analyze this problem.

Before getting to the simulations, let's consider the analytical probability (that is, the theoretical probability we can calculate.) As [Wikipedia explains](https://en.wikipedia.org/wiki/Birthday_problem#Calculating_the_probability), we want to calculate $p(n)$, the probability that at least two people in a room of $n$ people have the same birthday, but it turns out to be much easier to calculate ${\bar {p}}(n) = 1-p(n)$, the probability that *no* two people in the room have the same birthday (that is, the complement).

For example, with three people in a room, the probability that none of them have the same birthday is $$\bar{p}(3) = 1 \times \left(1-{\frac {1}{365}}\right) \times \left(1-{\frac {2}{365}}\right).$$ This follows from the fact that to require no matching birthdays we can allow the first person to have any birthday at all, while the second person can have any birthday *except* the one that the first person has, and the third person can have any birthday except either of the first two.

Generalizing to $n$ people, the probability is given by

$${\displaystyle {\begin{aligned}{\bar {p}}(n)&=1\times \left(1-{\frac {1}{365}}\right)\times \left(1-{\frac {2}{365}}\right)\times \cdots \times \left(1-{\frac {n-1}{365}}\right)\\&={\frac {365\times 364\times \cdots \times (365-n+1)}{365^{n}}}\\&={\frac {365!}{365^{n}(365-n)!}}={\frac {n!\cdot {\binom {365}{n}}}{365^{n}}}={\frac {_{365}P_{n}}{365^{n}},}\end{aligned}}}$$
where ${_{k}P_{n}}$ denotes the permutation function. (To get the second line, multiply every term by $\frac{365}{365}$.)

## Question 1

Write a function that takes $n$ as the argument and returns the analytical probability for $p(n) = 1-\bar{p}(n)$ using the formua above. The permutation function `perm` is available in `scipy.special`.

In [ ]:
# Import packages used in HW

from scipy.special import perm
import pandas as pd
import numpy as np
import random
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import pandas_datareader as pdr

In [ ]:
def p_bar_n(n):
    prob = 1 - perm(365,n)/(365**n)
    return prob

Test your function by evaluating it at $n=23$. Do you get the same number reported in the Wikipedia article (right under [equation 2](https://en.wikipedia.org/wiki/Birthday_problem#math_2))?

In [ ]:
p_bar_n(23)

Create a `np.array` with the probabilities for all values of $n$ from 1 to 100. Call it "probs".

In [ ]:
probs = np.fromiter((p_bar_n(n) for n in range(1,101)),float)

Use the code below to plot these probabilities.

In [ ]:
%matplotlib notebook

plt.plot(probs);

## Question 2

Now we'll use simulations to see if we can get the same results without using any math. Start by writing a function that takes arguments `n` and `num_trials` and estimates the probability through simulation. 

Each simulation should:
* Create a "room" with `n` people. What data structure will work best here? (list, dictionary, numpy array, pandas Series, etc.)
* Assign each person a random birthday. Practically speaking, you would probably just give them a number from 1 to 365.
* Check if any two people have the same birthday, and add one to a counter to keep track of how many simulations have met this condition

Call your function `birthday_sim`. It function should return the estimated probability (that is, the number of simulated rooms with a match, divided by the total number of rooms). A good default value for `num_trials` is 5000.

In [ ]:
def birthday_sim(n,num_trials = 5000):
    counter = 0
    for i in range(0,num_trials):
        simulation = np.unique(np.random.randint(1,366,n))
        if len(simulation) < n:
            counter += 1
        else:
            continue
            
    return counter / num_trials

Evaluate your function at $n=23$.

In [ ]:
birthday_sim(23)

Evaluate your function at $n=23$ using 100,000 trials.

In [ ]:
birthday_sim(23, num_trials = 100000)

Run the code below to draw a picture of the probabilities from the simulation. How similar is it to the graph generated from analytical probabilities earlier?

In [ ]:
# Try a bunch of inputs at once and store in a list
# To keep time manageable, don't do too many trials
probs = [birthday_sim(n, num_trials=5000) for n in range(1,50)]

# Plot
index = range(1,len(probs)+1)
plt.figure(figsize=(8, 6))
plt.bar(index, probs, alpha=0.5)
plt.title('Probability of Two People Having Same Birthday', fontsize=14)
plt.xlabel('Number of People in Room', fontsize=12)
plt.ylabel('Probability', fontsize=12);

In [ ]:
# The graph above is extremely similar to the graph generated from the analytical probabilities in question 1.
# Also, when ran to include 100 people in the room, the similarity remains. 

# Part 2 — Sentiment

## Question 3

Use the AAII survey data we used in class to see whether sentiment about future returns is more closely related to future returns or past returns. In particular, compare the correlations between the bull/bear spread and future 6-mo or 1-yr returns with the spread measure and *past* returns over the same horizons. Comment on what you learn from your findings.

In [ ]:
sent = pd.read_excel('http://www.aaii.com/files/surveys/sentiment.xls',
                     skiprows=7, header=None, usecols='A:D,M',
                     names = ['Date', 'Bullish', 'Neutral', 'Bearish', 'sp500'])

pd.to_datetime(sent['Date'], errors='coerce')
sent['Date'] = pd.to_datetime(sent['Date'], errors='coerce')
sent = sent.dropna(subset=['Date'])
sent = sent.set_index('Date')
sent['spread'] = sent['Bullish'] / sent['Bearish']
sent['p6mret'] = sent['sp500'] / sent['sp500'].shift(26) -1
sent['f6mret'] = sent['sp500'].shift(-26) / sent['sp500'] -1

sent = sent.dropna()

pearsonr(sent['spread'], sent['f6mret'])
sent['spread_sm'] = sent['spread'].ewm(alpha=.5).mean()
pearsonr(sent['spread_sm'], sent['f6mret'])
df = sent.loc['2001':]
for v in ['Bullish', 'Neutral', 'Bearish', 'spread']:
    y, z = pearsonr(df[v].ewm(alpha=.25).mean(), df['p6mret'])
    print('Correlation of {} and p6mret: {:.3f} ({:.2f})'.format(v, y, z) )
    r, p = pearsonr(df[v].ewm(alpha=.25).mean(), df['f6mret'])
    print('Correlation of {} and f6mret: {:.3f} ({:.2f})\n'.format(v, r, p))

In [ ]:
# Based on the outout above, we found that the sentiment data is primarily predicated
# on past performance rather than future (analyzing 6-month performance trends).
# Relatively spearking, the correlation between the bull-bear spread and past
# 6-month returns is stronger than the spread and future returns. Also, interestingly,
# there is a negative correlation between the spread and future returns. 
# Furthermore, the relative strength observation is also observed for bullish, neutral, and bearish.

# Part 3 — Returns

Begin with a DataFrame of S&P 500 member firms. You constructed this in a previous assignment. For this part, all you need is ticker symbols.

In [ ]:
html = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies', header = 0)
comps = html[0]['Symbol']
comps = pd.DataFrame(comps)

 ## Question 4

Select a random sample of 25 stocks. (Use a random seed so your results can be replicated.)

Download historical price data using the pandas datareader for each stock. By default, the datareader gets data beginning in 2010, which is fine for our purposes. Here's an example of downloading prices for IBM.

Construct a `DataFrame` in which each column contains daily *returns* (not prices) for a different stock in your sample. That is, it should have 25 columns.

In [ ]:
tickers = random.sample(list(set(comps['Symbol'])),25)
data = pdr.get_data_yahoo(tickers)
data = data.loc[:,['Adj Close']]
returns = data['Adj Close'].pct_change()
len(returns.columns)

For each stock, calculate the autocorrelation of returns. Note that the autocorrelation function is a method for `Series` but not `DataFrames`. You can apply it to every column using the `apply` function like this:

`df.apply(lambda x: x.autocorr())`.

How does the average autocorrelation compare to the autocorrelation of the S&P 500 index portfolio? (Ticker symbol: '^GSPC'). Try to explain any differences you observe.

In [ ]:
autocorr = returns.apply(lambda x: x.autocorr())
autocorr.describe()

In [ ]:
itick = '^GSPC'
idata = pdr.get_data_yahoo(itick)
iret = idata['Adj Close'].pct_change()
iframe = pd.DataFrame({itick:iret})
iframe = iframe.fillna(0)
iframe[itick].autocorr()

In [ ]:
# Based on the output, the S&P has a relatively stronger negative autocorrelation (absolute difference 
# depends on sample). Also, it is important to note that the S&P 500 index is market cap weighted with 
# 500 stocks, whereas the sample used is not adjusted for market cap and uses only 25 randomly chosen stocks 
# from the index. Therefore, we conclude that the difference in autocorrelation (index v. sample), as well
# as the large relative standard deviation for the sample values, is primarily due to the reasons previously 
# stated.